In [20]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fig,ax = plt.subplots(1,2)

ds0 = xr.open_dataset(f'../output/larmip_ctrl2/scalar_output_ANT_00001.nc')
time = np.arange(0,199)

runs = ['amun','wedd','ross','eais','pens']#,'all']
for r,run in enumerate(runs):
    #Read time series
    ds = xr.open_dataset(f'../output/larmip_{run}/scalar_output_ANT_00001.nc')
    SLR = ds0.ice_volume_af-ds.ice_volume_af #m
    IML = (ds0.ice_volume-ds.ice_volume)*1028*3.611e14*1e-12 #Gt
    ds.close()

    #Convert to response functions
    RS = (SLR[1:-1].values-SLR[:-2].values)/8 #m/yr /m/yr
    RI = (IML[1:-1].values-IML[:-2].values)/8 #Gt/yr / m/yr

    #Plot
    ax[0].plot(time,1000*RS,label=run)
    ax[1].plot(time,RI)

    #Save output
    dss = xr.Dataset()
    dss = dss.assign_coords({'time':time})
    dss['RF']       = (['time'],RS)
    dss['RF_total'] = (['time'],RI)
    dss['RF'].attrs['units'] = 'm/m'
    dss['RF_total'].attrs['units'] = 'Gt/m'
    dss.to_netcdf(f'../output/lrfs/LRF_{run}')
    dss.close()

ax[0].legend()
ds0.close()